<a href="https://colab.research.google.com/github/FlamingoLindo/Deep-Learnig-COVID-19-Radiography/blob/main/deep_learnig_covid_19_radiography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import IPython.display as display
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os,random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
! git clone https://github.com/FlamingoLindo/Deep-Learnig-COVID-19-Radiography.git

Cloning into 'Deep-Learnig-COVID-19-Radiography'...
remote: Enumerating objects: 21129, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 21129 (delta 1), reused 8 (delta 0), pack-reused 21117 (from 1)
Receiving objects: 100% (21129/21129), 738.78 MiB | 16.44 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (21168/21168), done.


In [3]:
IMAGE_HEIGHT=299
IMAGE_WIDTH=299
BATCH_SIZE=64
def get_pathframe(path):
  '''
  Obtém todos os caminhos das imagens e seus rótulos correspondentes e armazena
  no dataframe do pandas
  '''
  filenames = os.listdir(path)
  categories = []
  paths=[]
  for filename in filenames:
    paths.append(path+filename)
    category = filename.split('-')[0]
    if category == 'COVID':
      categories.append(0)
    elif category == 'Lung_Opacity':
      categories.append(1)
    elif category == 'Normal':
      categories.append(2)
    elif category == 'Viral Pneumonia':
      categories.append(3)

  df= pd.DataFrame({
      'filename': filenames,
      'category': categories,
      'paths':paths
  })
  return df

In [4]:
df=get_pathframe("/content/Deep-Learnig-COVID-19-Radiography/Dataset/")
df.head(5)

,filename,category,paths
0,Normal,2,/content/Deep-Learnig-COVID-19-Radiography/Dat...
1,COVID,0,/content/Deep-Learnig-COVID-19-Radiography/Dat...
2,Viral Pneumonia,3,/content/Deep-Learnig-COVID-19-Radiography/Dat...
3,Lung_Opacity,1,/content/Deep-Learnig-COVID-19-Radiography/Dat...


In [5]:
def load_and_preprocess_image(path):
  '''
  Carrega cada imagem e redimensiona para o formato desejado
  '''
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [IMAGE_WIDTH, IMAGE_HEIGHT])
  image /= 255.0  # normalize to [0,1] range
  return image

def convert_to_tensor(df):
  '''
  Converte cada dado e rótulo em um tensor
  '''
  path_ds = tf.data.Dataset.from_tensor_slices(df['paths'])
  image_ds = path_ds.map(load_and_preprocess_image)
  # onehot_label=tf.one_hot(tf.cast(df['category'], tf.int64),2) if using softmax
  onehot_label=tf.cast(df['category'], tf.int64)
  label_ds = tf.data.Dataset.from_tensor_slices(onehot_label)

  return image_ds,label_ds

In [6]:
X,Y=convert_to_tensor(df)
print("Shape of X in data:", X)
print("Shape of Y in data:", Y)

Shape of X in data: <_MapDataset element_spec=TensorSpec(shape=(299, 299, 3), dtype=tf.float32, name=None)>
Shape of Y in data: <_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


In [7]:
dataset=tf.data.Dataset.zip((X,Y)).shuffle(buffer_size=21165)
dataset_train=dataset.take(14815)
dataset_test=dataset.skip(6350)

dataset_train=dataset_train.batch(BATCH_SIZE, drop_remainder=True)
dataset_test=dataset_test.batch(BATCH_SIZE, drop_remainder=True)
dataset_train

<_BatchDataset element_spec=(TensorSpec(shape=(64, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(64,), dtype=tf.int64, name=None))>

In [8]:
def plotimages(imagesls):
  fig, axes = plt.subplots(1, 5, figsize=(20,20))
  axes = axes.flatten()
  for image,ax in zip(imagesls, axes):
    ax.imshow(image)
    ax.axis('off')

imagesls=[]
for n, image in enumerate(X.take(5)):
  imagesls.append(image)

plotimages(imagesls)

FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:1 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::ParallelMapV2: /content/Deep-Learnig-COVID-19-Radiography/Dataset/Normal; Is a directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

In [ ]:
def My_CNNmodel():

  model = tf.keras.models.Sequential()
  model.add(layers.Conv2D(8, (3, 3), padding='same',activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Conv2D(16, (3, 3), padding='same',activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Conv2D(32, (3, 3), padding='same',activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid')) # softmax

  opt=tf.keras.optimizers.Adam(0.001)
  model.compile(optimizer=opt,
              loss='binary_crossentropy', # loss='categorical_crossentropy' if softmax
              metrics=['accuracy'])

  return model

In [ ]:
model=My_CNNmodel()
model.summary()

In [ ]:
hist = model.fit(dataset_train, epochs=10, validation_data=dataset_test)

In [ ]:
model.save("my_model.keras")

In [ ]:
def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))

    # Plot accuracy
    axs[0].plot(range(1, len(model_history.history[acc]) + 1), model_history.history[acc])
    if val_acc in model_history.history:
        axs[0].plot(range(1, len(model_history.history[val_acc]) + 1), model_history.history[val_acc])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['train', 'val'] if val_acc in model_history.history else ['train'], loc='best')

    # Plot loss
    axs[1].plot(range(1, len(model_history.history['loss']) + 1), model_history.history['loss'])
    if 'val_loss' in model_history.history:
        axs[1].plot(range(1, len(model_history.history['val_loss']) + 1), model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['train', 'val'] if 'val_loss' in model_history.history else ['train'], loc='best')

    plt.show()

plot_model_history(hist)

In [ ]:
tf.keras.layers.Dropout(0.5)

In [ ]:
tf.keras.layers.BatchNormalization()

In [ ]:
# Predict the values from the validation dataset
loss, accuracy = model.evaluate(dataset_test)
print("Test: accuracy = %f  ;  loss = %f " % (accuracy, loss))
y_pred =  model.predict(dataset_test)
y_p = np.where(y_pred > 0.5, 1,0)
test_data=dataset_test.unbatch()
y_g=[]
for image, label in  test_data:
  y_g.append(label.numpy())

confusion_mtx = confusion_matrix(y_g, y_p)
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Blues",linecolor="gray", fmt= '.1f',ax=ax,
            xticklabels=["Gatos", "Cachorros"], yticklabels=["Gatos", "Cachorros"])
plt.xlabel("Rótulo Previsto")
plt.ylabel("Rótulo Verdadeiro")
plt.title("Matriz de Confusão")
plt.show()

In [ ]:
report = classification_report(y_g, y_p, target_names=['Gatos','Cachorros'])
print(report)